# Data Processing
---
### First inspection of dataset, data filtering and cleaning

Creation: 05.02.2021

## Required Libraries
---

In [ ]:
import pandas as pd # used to store the datasets
import numpy as np  # used for numerical calculations on individual columns

## Constants

In [ ]:
# path lookup dictionary to store the relative paths from the directory containing the jupyter notebooks to important directories in the project
PATH = {}
PATH["data_raw"] = "../data/raw/"
PATH["data_interim"] = "../data/interim/"
PATH["data_processed"] = "../data/processed/"
PATH["data_external"] = "../data/external/"
PATH["references"] = "../data/references"

# filename lookup dictionary storing the most relevant filenames
FILENAME = {}
FILENAME["accidents"] = "Road Safety Data - Accidents 2019.csv"
FILENAME["casualties"] = "Road Safety Data - Casualties 2019.csv"
FILENAME["vehicles"] = "Road Safety Data- Vehicles 2019.csv" # the original dataset has a small typing mistake
FILENAME["variable_lookup"] = "variable lookup.xls"

DATA_RAW = {}

DATA_LEEDS = {}

# list of internal names for datasets
TABLENAMES = ["accidents", "casualties", "vehicles"]

## Loading in the Dataset
---
Starting of the data analysis, we import the given three datasets into a 'pandas' dataframe. 

In [ ]:
# load all three datasets using pandas into the predefined dictionary 'DATA_RAW' where the key corresponds to the internal dataset name
for name in TABLENAMES:
    DATA_RAW[name] = pd.read_csv(PATH['data_raw'] + FILENAME[name])

## Initial Sanity Check
---
Before continuing with the data analysis, we want to make sure that the dataset is clean. We do this in a several ways. 
- (a) Are there multiple indexes in 'accidents.csv'
- (b) We check if there are indexes in the sub datasets 'raw_data_vehicles' and 'raw_data_casualties' that are not in the central dataset 'raw_accidents.csv'
- (c) We check the missing values in each column

### Mulitple Indexes
We are first evaluating if there are mulitple indexes in the big dataset 'accidents.csv'. This is a first, very basic metric to determine whether the data in the dataset has been inputted correctly. In this case it is even more important since the accident indexes link the main dataset 'accidents.csv' to the two sub datasets.

In [ ]:
# here we check if there are mulitple indexes in the accidents dataset
DATA_RAW['accidents'].shape[0] == len(set(DATA_RAW['accidents']['Accident_Index']))

Perfect. There do not seem to be any multiple indexes. Each row in the dataset 'accidents.csv' seems to refer to a unique accident that we can reference in the two sub datasets.

### Wrong Indexes in Sub-Datasets
It would be bad if there is information on vehicles and casualties in the two subsets that are referenced by an Accident ID that is not in the main dataset 'accidents.csv'. So we check for those using the below code.

In [ ]:
def check_indexes_in_subset(sub_dataset, _in=DATA_RAW['accidents']):
    """ 
    Helper-Function to evaluate whether there are indexes in the two linked sub datasets that do not appear in the main dataset.

    Parameters:
        sub_dataset         : pd.DataFrame
        _in                 : pd.DataFrame
    Return:
        #Wrong Indexes      : int (None if len() == 0)
    """
    
    accidents_indexes = set(_in['Accident_Index'])
    wrong_indexes = [i for i in set(sub_dataset['Accident_Index']) if i not in accidents_indexes]

    if len(wrong_indexes) == 0:
        return None
    else:
        return len(wrong_indexes)

In [ ]:
# computing the wrong indexes for each sub dataset
for dataset in TABLENAMES:
    if dataset != 'accidents':
        print(f"#Missing indexes in {dataset.capitalize()}: {check_indexes_in_subset(DATA_RAW[dataset], DATA_RAW['accidents'])}")

"This is rather bad. Roughly 21.500 indexes in the 'vehicles.csv' raw dataset are linking to an accident that is not registered in the 'accidents.csv', 19.300 in the 'casualties.csv' are linking casualties to accidents that are not registered int the 'accidents.csv'. If a similar behavior can be observed in the dataset that is filtered for Leeds, we have to think about how to deal with those obvious wrong input of data. 

### Check for missing values
Here we check in all columns of all datasets to see if there are missing values (empty string) to get a feeling on which columns we need to further do processing.

In [ ]:
def check_column_for_missing_value(column):
    """ 
    For a column of a pd.DataFrame (One-Dimensional Array) the fucntion returns the number of Null-Values that correspond to the          number of missing values in the column.

    Parameters:
        column              : pd.DataFrame
    Return:
        #Null Values        : int
    """

    return sum(column.isnull())

In [ ]:
def check_all_columns(data):
    """
    For a dataset provided as a pd.DataFrame the function returns an informative string about each column containing null values,         namely the number of missing values, the column index and the variable name of the column.

    Parameters:
        data                : pd.DataFrame
    Return:
        Informative String for each column containing null values, else None
    """

    for column in range(data.shape[1]):
        if check_column_for_missing_value(data.iloc[:,column]) != 0:
            print(f'{check_column_for_missing_value(data.iloc[:,column])} ({data.columns[column]}({column}))')

In [ ]:
for dataset in TABLENAMES:
    print(dataset.capitalize())
    check_all_columns(DATA_RAW[dataset])
    print('\n')

That's not bad! There are no missing values in both sub datasets. 
It seems like in the accidents dataset, there are 28 accidents that have no information on their location. This only gets important for our analysis if one of those accidents is located in Leeds, then we would need to deal with this issue later. The LSOA Metric - which is another measure of the accident location of the accident - hasn't been registered for 5714 accidents. This is not important for our analysis, since we will use the longitude and latitude to plot the accidents' location. 
There are, however, 63 accidents for which the time of accident is not registered. If any of those accidents are located in Leeds, we have to deal with them later.

## Filtering 
---
Next, we filter the main dataset 'accidents.csv' for the city of interest 'Leeds', which can be identified by several variables in the dataset. We here chose the column 'Local Authority (District)', where 'Leeds' is identified as 204. The resulting, filtered dataframe is saved into a new variable.

In [ ]:
DATA_LEEDS["accidents"] = DATA_RAW['accidents'][DATA_RAW['accidents']['Local_Authority_(District)'] == 204]

However, the other two datasets cannont be identified by the variable attributes, but need to be filtered through the unique accident indexes that we can obtain from our filtered dataframe of accidents in 'Leeds'. We obtain a list of all accident indexes of the accidents that occured in Leeds and use this index list to filter both the 'vehicles.csv' and 'casualties.csv' datasets.

In [ ]:
leeds_indexes = list(DATA_LEEDS['accidents']['Accident_Index']) # we can do it with list because we know that all indexes are unique

In [ ]:
DATA_LEEDS["casualties"] = DATA_RAW['casualties'][DATA_RAW['casualties']['Accident_Index'].isin(leeds_indexes)]
DATA_LEEDS["vehicles"] = DATA_RAW['vehicles'][DATA_RAW['vehicles']['Accident_Index'].isin(leeds_indexes)]

## Saving Filtered Data
---

In [ ]:
for dataset in TABLENAMES:
    DATA_LEEDS[dataset].to_csv(PATH['data_interim'] + FILENAME[dataset], index=False)

## Sanity Check for Leeds
--- 
Now, that we filtered the dataset, we do the exact same snaity checks that we performed on the raw datasets.

### Mulitple Indexes

In [ ]:
DATA_LEEDS['accidents'].shape[0] == len(set(DATA_LEEDS['accidents']['Accident_Index']))

### Wrong Indexes in Sub-Datasets

In [ ]:
for dataset in TABLENAMES:
    if dataset != 'accidents':
        print(f"#Missing indexes in {dataset.capitalize()}: {check_indexes_in_subset(DATA_LEEDS[dataset], DATA_LEEDS['accidents'])}")

### Missing Values

In [ ]:
for dataset in TABLENAMES:
    print(dataset.capitalize())
    check_all_columns(DATA_LEEDS[dataset])
    print('\n')

Perfect. None of the sanity checks reports any problems on our dataset. At this point we could export the dataset and work on the filtered ones. However, we are making some adjustments in the below section to make our analysis easier.

## Process Data
---
In this section, the 'Date' and 'Time' attributes in the 'accidents.csv' module will be cleaned for easy use in the single variable analysis.

### Time

In [ ]:
time = np.array(DATA_LEEDS['accidents']['Time'])
for i in range(len(time)):
    try: 
        time[i] = time[i][:2]
    except:
        time[i] = '-1'

DATA_LEEDS['accidents']['Time'] = time

### Date

In [ ]:
date = np.array(DATA_LEEDS['accidents']['Date'])


for i in range(len(date)):
    date[i] = int(date[i][3:5])

DATA_LEEDS['accidents']['Date'] = date

### 2nd Road Class
---

In [ ]:
second_road_class = np.array(DATA_LEEDS['accidents']['2nd_Road_Class'])

for i in range(len(second_road_class)):
    if second_road_class[i] == -1:
        second_road_class[i] = 0

DATA_LEEDS['accidents']['2nd_Road_Class'] = second_road_class

## Overview 
---
For each of the datasets, we want to get a first good impression of its size and the information it stores. To gain this information, we print out each of the datasets, and get a summary of each of the columns and the uniques.

### Accidents
---

In [ ]:
DATA_LEEDS['accidents'].shape # prints out the number of columns and rows

In [ ]:
DATA_LEEDS['accidents'] # prints out an overview of the dataframe (and the number of rows and columns)

In [ ]:
DATA_LEEDS['accidents'].nunique() # prints out the column names and the corresponding number of unique values 

We see, that the main dataset 'accidents_processed.csv' stores all recorded accidents in 2019 in Leeds. It consist of 1451 columns (which leads to 1450 recorded accidents) and has 32 columns providing more detailed information about the accident. The different variables and the number of its unique values can be studied in the output of the above cell. We see, that we can differentiate the attributes as follows:
- Categorical Attributes (Most of the columns are categorical)
- Geographical Attributes (There are several measures of the location of the accident)
- Time Attribute (Each accident specifies a date and time)

### Vehicles
---

In [ ]:
DATA_LEEDS['vehicles'].shape # prints out the number of columns and rows

In [ ]:
DATA_LEEDS['vehicles'] # prints out an overview of the dataframe (and the number of rows and columns)

In [ ]:
DATA_LEEDS['vehicles'].nunique() # prints out the column names and the corresponding number of unique values 

We see, that the side dataset 'vehicles_processed.csv' provides more detailed information about all vehicles involved in each of the accidents. It consist of 2688 columns (which leads to 2688 records on involved vehicles) and has 23 columns providing more detailed information about the vehicle. The different variables and the number of its unique values can be studied in the output of the above cell. We see, that we can differentiate the attributes as follows:
- Linking Attributes (Accident Indexes link the vehicles to the accidents dataset and the vehicle references the casualties)
- Categorical Attributes (Most of the columns are categorical)

### Casualties
---

In [ ]:
DATA_LEEDS['casualties'].shape # prints out the number of columns and rows

In [ ]:
DATA_LEEDS['casualties'] # prints out an overview of the dataframe (and the number of rows and columns)

In [ ]:
DATA_LEEDS['casualties'].nunique() # prints out the column names and the corresponding number of unique values 

We see, that the side dataset 'casualties_processed.csv' provides more detailed information about the casualties of all lethal accidents. It consist of 1908 columns (which leads to 1907 records on casualties) and has 16 columns providing more detailed information about the vehicle. The different variables and the number of its unique values can be studied in the output of the above cell. We see, that we can differentiate the attributes as follows:
- Linking Attributes (Accident Indexes link the vehicles to the accidents dataset and the vehicle references the casualties)
- Categorical Attributes (Most of the columns are categorical)

## Export Processed Datasets
--- 
Finally, we export the processed datasets into a new subfolder. From now on, all Jupyter Notebooks will work with those processed datasets.

In [ ]:
for dataset in TABLENAMES:
    DATA_LEEDS[dataset].to_csv(PATH['data_processed'] + FILENAME[dataset], index=False)